In [5]:
import json
import os
import random
import re
import time
from datetime import datetime
import hashlib
# openai 라이브러리 v1.17.0 이상 권장
from openai import OpenAI, APIError
# pydantic v2 이상 권장! (pip install -U pydantic)
from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Any, Optional, Union, Type, Tuple

# --- 0. 환경 설정 ---
# !!! 실행 전 환경변수 'FRIENDLI_TOKEN'에 API 키를 설정해야 합니다 !!!
if "FRIENDLI_TOKEN" not in os.environ:
    raise ValueError("환경변수 'FRIENDLI_TOKEN'이 설정되지 않았습니다.")

client = OpenAI(
    base_url="https://api.friendli.ai/serverless/v1",
    api_key=os.environ.get("FRIENDLI_TOKEN")
)
# !!! 모델 ID 확인 !!!
MODEL_ID = "deepseek-r1" # 일반 대화, Blueprint 생성 등에 사용할 모델
FC_MODEL_ID = "deepseek-r1" # Function Calling 시 사용할 모델

# --- Pydantic 모델 정의 ---
class ToolCallInternal(BaseModel):
    tool_name: str = Field(..., description="호출할 도구의 이름")
    arguments: Dict[str, Any] = Field(..., description="도구 호출에 필요한 인자들 (딕셔너리 형태)")

class GetWeatherArgs(BaseModel):
    location: str = Field(..., description="날씨를 조회할 도시 이름")

class GetNewsArgs(BaseModel):
    topic: str = Field(..., description="뉴스를 검색할 주제")

class Blueprint(BaseModel):
    q: str = Field(..., description="사용자의 초기 질문 또는 요청")
    a_gt: List[ToolCallInternal] = Field(..., description="Ground Truth: 에이전트가 수행해야 할 도구 호출 순서")
    o_gt: str = Field(..., description="예상되는 최종 결과에 대한 설명 (문자열)")

# ---------------

# --- 도구 정의 (Fake 데이터 반환) ---
def get_weather(location: str) -> str:
    """주어진 location(도시명)의 오늘 날씨 정보를 반환합니다."""
    print(f"** 도구 실행: get_weather(location='{location}') - Fake Data 반환 **")
    fake_weather_data = {"location": location, "date": "2025-01-01", "weather": "항상 맑음", "temperature": "23°C"}
    return json.dumps(fake_weather_data, ensure_ascii=False)

def get_news(topic: str) -> str:
    """주어진 topic(뉴스 주제)의 최신 뉴스 헤드라인을 반환합니다."""
    print(f"** 도구 실행: get_news(topic='{topic}') - Fake Data 반환 **")
    fake_news_data = {"topic": topic, "date": "2025-01-01", "headlines": ["고정 뉴스 A", "고정 뉴스 B"]}
    return json.dumps(fake_news_data, ensure_ascii=False)

available_tools_mapping = {"get_weather": get_weather, "get_news": get_news}
tools_definition_for_api = [
    {"type": "function", "function": {"name": "get_weather", "description": "지정된 위치의 현재 날씨 정보를 가져옵니다.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "날씨를 조회할 도시 이름 (예: 서울, Paris)"}}, "required": ["location"]}}},
    {"type": "function", "function": {"name": "get_news", "description": "지정된 주제에 대한 최신 뉴스 기사 몇 개를 가져옵니다.", "parameters": {"type": "object", "properties": {"topic": {"type": "string", "description": "뉴스를 검색할 주제 (예: IT, 경제)"}}, "required": ["topic"]}}},
]

# ---------------

# --- Helper Function: API 호출 ---
def call_friendli_api(messages: List[Dict[str, Any]], tools: Optional[List[Dict[str, Any]]] = None, tool_choice: Optional[Union[str, Dict[str, Any]]] = None, role_description: str = "API Call", max_retries: int = 1) -> Optional[Dict[str, Any]]:
    """ Friendli.ai API를 호출하는 중앙 함수 """
    print(f"\n--- Friendli API 호출 ({role_description}) ---")
    last_exception = None
    current_model_id = FC_MODEL_ID if tools else MODEL_ID
    print(f"사용 모델: {current_model_id}")
    for attempt in range(max_retries + 1):
        try:
            params = {"model": current_model_id, "messages": messages}
            if tools: params["tools"] = tools
            if tool_choice: params["tool_choice"] = tool_choice
            completion = client.chat.completions.create(**params)
            response_message = completion.choices[0].message
            log_content = response_message.content or ""
            log_tool_calls = f" Tool Calls: {[tc.function.name for tc in response_message.tool_calls]}" if response_message.tool_calls else ""
            print(f"성공: 응답 Role={response_message.role} Content='{log_content[:50]}...' {log_tool_calls}")
            return response_message.model_dump()
        except APIError as e:
            print(f"시도 {attempt + 1}: API 오류: {e}")
            last_exception = e
            if hasattr(e, 'status_code') and e.status_code == 422: print("422 에러 발생. 재시도 중단."); break
            if attempt < max_retries: time.sleep(1.5 ** attempt)
        except Exception as e:
            print(f"시도 {attempt + 1}: 예외 발생: {e}")
            last_exception = e
            if attempt < max_retries: time.sleep(1.5 ** attempt)
    print(f"최대 재시도({max_retries}) 실패. 오류: {last_exception}"); return None

# ---------------

# --- Helper Function: Blueprint ID 생성 ---
def create_blueprint_id(blueprint: Blueprint) -> str:
    """Blueprint 객체로부터 고유 식별 해시를 생성합니다."""
    try:
        a_gt_serializable = sorted([{"tool_name": tc.tool_name, "arguments": dict(sorted(tc.arguments.items()))} for tc in blueprint.a_gt], key=lambda x: (x['tool_name'], json.dumps(x['arguments'], sort_keys=True)))
        data_to_hash = {"q": blueprint.q, "a_gt": a_gt_serializable, "o_gt": blueprint.o_gt}
        canonical_string = json.dumps(data_to_hash, sort_keys=True, ensure_ascii=False)
        hash_object = hashlib.sha256(canonical_string.encode('utf-8')); return hash_object.hexdigest()
    except Exception as e: print(f"오류: Blueprint ID 생성 중 예외: {e}"); return f"error_{random.random()}_{time.time()}"

# ---------------

# --- Helper Function: 응답에서 <think> 태그 제거 ---
def remove_think_tags(text: Optional[str]) -> Optional[str]:
    """주어진 텍스트에서 <think>...</think> 블록을 제거합니다."""
    if text is None:
        return None
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

# ---------------

# --- Step 1-1: 청사진 생성 (LLM 호출) ---
def generate_blueprint_string(tools_api_def: List[Dict[str, Any]], max_retries: int = 2) -> Optional[str]:
    """LLM을 호출하여 작업 청사진 JSON 문자열을 생성합니다."""
    print("\n--- 1-1: 작업 청사진 생성 (LLM 호출) ---")
    tool_desc_str = "\n".join([f"- {t['function']['name']}: {t['function'].get('description', '')}" for t in tools_api_def])
    # **** 시스템 프롬프트 수정 ****
    system_prompt_gen = f"""당신은 APIGen-MT 파이프라인의 1단계를 위한 '작업 청사진 생성자'입니다. 당신의 목표는 사용자와 AI 에이전트 간의 **현실적이고 검증 가능한 다중 턴 상호작용 시나리오**를 나타내는 상세한 청사진을 만드는 것입니다. 주어진 도구 설명을 바탕으로 다음 구성요소를 포함하는 **JSON 객체**를 생성해야 합니다.

    1.  `q` (string): 사용자의 초기 질문/요청입니다. **구체적이고 자연스러워야 하며**, 에이전트가 정보를 조회하고, 상태를 변경하고, 사용자에게 다시 확인하는 등 **여러 단계의 상호작용이 필요할 수 있는** 시나리오를 나타내는 것이 좋습니다.
    2.  `a_gt` (list): 사용자의 요청 `q`를 **완전히, 그리고 정확한 순서대로** 해결하기 위해 에이전트가 호출해야 하는 **Ground Truth 도구 호출 리스트**입니다. 각 요소는 `{{"tool_name": "도구명", "arguments": {{"인자명": "값", ...}}}}` 형식이어야 합니다. 사용 가능한 도구 설명과 반드시 일치해야 합니다. **최소 1개 이상의 도구 호출**을 포함해야 하며, 가능하면 **2개 이상의 순차적 도구 호출**이 필요한 시나리오를 만드세요.
    3.  `o_gt` (string): 모든 `a_gt`가 성공적으로 실행되었다고 가정했을 때, 에이전트가 사용자에게 최종적으로 제공해야 할 **결과 요약 또는 응답 메시지에 대한 자연스러운 설명**입니다. `a_gt`의 모든 결과를 반영해야 합니다.

    **사고 과정 (Optional but Recommended):**
    - 최종 JSON을 생성하기 전에, 당신의 **사고 과정 (어떤 시나리오를 구상했고, 왜 특정 도구와 순서를 선택했는지 등)을 `<think>...</think>` 태그 안에 작성**할 수 있습니다.

    **응답 규칙:**
    - 최종 응답에는 반드시 위에 설명된 구조를 가진 **유효한 JSON 객체**가 ```json ... ``` 코드 블록 안에 포함되어야 합니다.
    - 코드 블록 앞이나 뒤에 **JSON 객체 외의 다른 텍스트는 포함하지 마십시오** (단, 먼저 나오는 `<think>` 블록은 허용됩니다)."""
    user_prompt_gen = f"사용 가능한 도구:\n{tool_desc_str}\n\n위 도구들을 사용하는 시나리오를 위한 JSON 객체를 위의 요구사항과 규칙에 맞게 생성해주세요."

    api_response = call_friendli_api(messages=[{"role": "system", "content": system_prompt_gen}, {"role": "user", "content": user_prompt_gen}], role_description="작업 생성 LLM", max_retries=max_retries)
    response_content = api_response.get("content") if api_response else None
    if not response_content: print("LLM 청사진 응답 받기 실패."); return None

    # JSON 블록 추출 (앞의 <think> 태그 등 무시)
    json_match = re.search(r'\{[\s\S]*\}', response_content)
    if json_match: print("성공: 청사진 JSON 문자열 추출 완료"); return json_match.group(0)
    else: print(f"오류: LLM 응답 JSON 찾기 실패. Raw: {response_content}"); return None

# ---------------

# --- Step 1: 청사진 파싱 및 검증 (Pydantic) ---
def parse_and_validate_blueprint(blueprint_str: str) -> Optional[Blueprint]:
    """JSON 문자열을 파싱하고 Blueprint Pydantic 모델로 검증합니다."""
    print("\n--- 1-1b: 청사진 파싱 및 검증 (Pydantic) ---")
    parsed_dict = None
    try:
        parsed_dict = json.loads(blueprint_str)
        blueprint_obj = Blueprint.model_validate(parsed_dict)
        print("성공: Blueprint 파싱 및 검증 완료")
        return blueprint_obj
    except Exception as e: print(f"Blueprint 파싱/검증 오류: {e}"); return None

# ---------------

# --- Step 1-2: 청사진 실행 가능성 검사 ---
def check_blueprint_execution(blueprint: Blueprint, tool_mapping: Dict[str, callable]) -> bool:
    """Blueprint의 a_gt 도구 호출 실행 가능성 검사."""
    print("\n--- 1-2: 형식/실행 검사 ---")
    for action in blueprint.a_gt:
        tool_name = action.tool_name; arguments = action.arguments
        if tool_name not in tool_mapping: print(f"오류: 도구 '{tool_name}' 없음"); return False
        try:
            if tool_name == "get_weather": GetWeatherArgs.model_validate(arguments)
            elif tool_name == "get_news": GetNewsArgs.model_validate(arguments)
        except ValidationError as e: print(f"오류: '{tool_name}' 인자 검증 실패: {e}"); return False
        print(f"도구 '{tool_name}' (인자: {arguments}) 실행 가능 확인.")
    print("형식/실행 검사 통과."); return True

# ---------------

# --- Step 1-3: 청사진 검토 (LLM 호출) ---
def review_blueprint(blueprint: Blueprint, max_retries: int = 1) -> Tuple[bool, str]:
    """LLM을 호출하여 Blueprint를 검토하고 결과(통과여부, 이유)를 반환합니다."""
    print("\n--- 1-3: 검토 위원회 ---")
    # **** 시스템 프롬프트 수정 ****
    system_prompt_review = """당신은 APIGen-MT 파이프라인의 '청사진 검토 위원회' 역할입니다. 제공된 작업 청사진({q, a_gt, o_gt})의 **논리적 타당성**과 **사용자 의도와의 정렬(alignment)**을 엄격하게 평가해야 합니다.

    **검토 기준:**
    1.  **의도-행동 일치:** 계획된 행동(`a_gt`)이 사용자의 질문(`q`)을 **정확하고, 완전하게, 그리고 효율적으로** 해결합니까? 불필요하거나 누락된 단계는 없습니까? 인자는 정확합니까?
    2.  **행동-결과 일치:** 예상 결과(`o_gt`)가 계획된 행동(`a_gt`)의 **논리적이고 그럴듯한(plausible) 결과**를 반영합니까? `a_gt`의 모든 중요한 결과가 `o_gt`에 요약되어 있습니까?
    3.  **전체 일관성:** 질문, 행동, 결과가 전체적으로 **자연스럽고 의미론적으로 일관성** 있습니까? 현실적인 시나리오입니까?

    **사고 과정 (Optional but Recommended):**
    - 최종 평가 전에, 각 기준에 대한 당신의 **판단 근거를 `<think>...</think>` 태그 안에 상세히 기술**할 수 있습니다.

    **응답 규칙:**
    - 모든 사고 과정 후에, 최종 평가 결과를 **반드시 'pass' 또는 'fail'로 시작하는 한 줄**로 명확히 작성해주세요.
    - 'pass' 또는 'fail' 결정 뒤에는 **간결하고 명확한 이유**를 덧붙여야 합니다.
    - 최종 평가 라인에는 `<think>` 태그가 포함되지 않아야 합니다. (예: `pass: 계획이 질문 의도와 완벽히 일치하며 모든 단계가 논리적임.`)"""
    user_prompt_review = f"다음 작업 청사진 검토:\n질문(q): {blueprint.q}\n액션(a_gt): {blueprint.model_dump_json(indent=2)}\n결과(o_gt): {blueprint.o_gt}\n\n타당한가요? (pass/fail 및 이유)"

    api_response = call_friendli_api(messages=[{"role": "system", "content": system_prompt_review}, {"role": "user", "content": user_prompt_review}], role_description="검토 위원회 LLM", max_retries=max_retries)
    review_response_content = api_response.get("content") if api_response else None

    review_passed = False; reason = "응답 없음/분석 불가"
    if review_response_content:
        # <think> 태그 제거 후 실제 결정/이유 텍스트 추출
        cleaned_response = remove_think_tags(review_response_content)
        if cleaned_response: # Think 태그만 있었던 경우 방지
             if re.search(r'^\W*\bpass\b', cleaned_response, re.IGNORECASE):
                 review_passed = True
                 match = re.search(r'^\W*\bpass\b\W*', cleaned_response, re.IGNORECASE)
                 reason = cleaned_response[(match.end() if match else 0):].strip() or cleaned_response
             else: # pass가 아니면 실패로 간주, 전체 cleaned 응답을 이유로
                 reason = cleaned_response
        else:
             reason = "Think 태그만 포함된 응답" # Think 태그 제거 후 내용이 없는 경우

    if review_passed: print(f"검토 통과. 이유: {reason}")
    else: print(f"검토 실패: {reason}")
    return review_passed, reason

# ---------------

# --- Step 2-1: 첫 사용자 턴 시뮬레이션 ---
def simulate_first_user_turn(blueprint_q: str, max_retries: int = 1) -> Optional[str]:
    """사용자의 첫 발화를 시뮬레이션합니다."""
    print("\n--- 2-1: 시뮬레이션된 인간 (첫 발화) ---")
    # 간단 버전: Blueprint의 q 사용 (LLM 호출 줄이기 위해)
    user_utterance = blueprint_q
    print(f"사용자 첫 발화 (Blueprint q 사용): {user_utterance}")
    return user_utterance
    # LLM 사용 버전 (필요시 아래 주석 해제 및 시스템 프롬프트 사용)
    # system_prompt_human1 = """당신은 특정 목표를 가진 사용자입니다. 주어진 목표(사용자 요청)를 AI 비서에게 전달하려고 합니다. 실제 사람이 AI에게 도움을 요청하는 것처럼 **매우 자연스럽고 구체적인 첫 번째 대화 메시지**를 생성해주세요. 한두 문장 정도로, 너무 딱딱하거나 간결하지 않게 작성하세요. 요청 내용을 그대로 복사하지 마세요."""
    # user_prompt_human1 = f"다음 요청을 AI 비서에게 전달하려고 합니다: '{blueprint_q}'. 어떻게 말을 시작하시겠어요?"
    # api_response = call_friendli_api(messages=[{"role": "system", "content": system_prompt_human1}, {"role": "user", "content": user_prompt_human1}], role_description="시뮬레이션된 인간 (첫 발화)", max_retries=max_retries)
    # raw_utterance = api_response.get("content") if api_response else None
    # user_utterance = remove_think_tags(raw_utterance) # Think 태그 제거
    # if user_utterance: print(f"사용자 첫 발화: {user_utterance}"); return user_utterance
    # else: print("사용자 첫 발화 생성/파싱 실패."); return None

# ---------------

# --- Step 2-2: 에이전트 턴 시뮬레이션 (Function Calling) ---
def simulate_agent_turn(history: List[Dict], tools_api_def: List[Dict], max_retries: int = 2) -> Optional[Dict[str, Any]]:
    """Function Calling을 사용하여 에이전트의 응답(텍스트 또는 도구 호출)을 생성합니다."""
    print("\n--- 2-2: 테스트 에이전트 (결정/응답) ---")
    # history의 첫 메시지가 에이전트 역할을 정의하는 system 메시지여야 함
    agent_response_message = call_friendli_api(messages=history, tools=tools_api_def, tool_choice="auto", role_description="테스트 에이전트 (결정)", max_retries=max_retries)
    if not agent_response_message: print("에이전트 응답 생성 실패.")
    return agent_response_message

# ---------------

# --- Step 2-3: 도구 실행 ---
def execute_tool_calls(tool_calls: List[Dict], tool_mapping: Dict[str, callable]) -> Tuple[List[Dict[str, Any]], List[ToolCallInternal]]:
    """모델 요청 도구 호출 실행 및 결과/성공 목록 반환."""
    tool_response_messages = []
    successfully_executed_calls = []
    for tool_call in tool_calls:
        tool_call_id = tool_call["id"]
        function_to_call = tool_call["function"]
        tool_name = function_to_call["name"]
        arguments_str = function_to_call["arguments"]
        print(f"\n--- 2-3: 환경/도구 실행기 ({tool_name}) ---")
        content = ""; executed_successfully = False; arguments_dict = None
        if tool_name in tool_mapping:
            try:
                arguments_dict = json.loads(arguments_str)
                if tool_name == "get_weather": GetWeatherArgs.model_validate(arguments_dict)
                elif tool_name == "get_news": GetNewsArgs.model_validate(arguments_dict)
                content = tool_mapping[tool_name](**arguments_dict)
                executed_successfully = True
            except Exception as e: print(f"오류: '{tool_name}' 실행/검증 실패: {e}"); content = json.dumps({"error": f"Tool error: {e}"})
        else: print(f"오류: 알 수 없는 도구 '{tool_name}'"); content = json.dumps({"error": f"Tool '{tool_name}' not found."})
        tool_response_messages.append({"role": "tool", "tool_call_id": tool_call_id, "name": tool_name, "content": content})
        if executed_successfully and arguments_dict is not None:
             successfully_executed_calls.append(ToolCallInternal(tool_name=tool_name, arguments=arguments_dict))
    return tool_response_messages, successfully_executed_calls

# ---------------

# --- Step 2-4: 최종 에이전트 응답 생성 ---
def get_final_agent_response(history_with_tool_results: List[Dict], tools_api_def: List[Dict], max_retries: int = 1) -> Optional[Dict[str, Any]]:
    """도구 결과를 포함한 히스토리 기반으로 최종 텍스트 응답 생성."""
    print("\n--- 2-4: 테스트 에이전트 (도구 결과 기반 응답 생성) ---")
    final_response = call_friendli_api(messages=history_with_tool_results, tools=tools_api_def, tool_choice="none", role_description="테스트 에이전트 (최종 응답)", max_retries=max_retries)
    if not final_response or final_response.get("content") is None: # content가 null 또는 없는 경우 체크
        print("경고: 최종 응답 생성 실패 또는 내용 없음.")
        return {"role": "assistant", "content": "알겠습니다. 요청하신 정보를 처리했습니다."} # Fallback
    # content가 빈 문자열("")일 수 있으므로 None만 체크
    return final_response

# ---------------

# --- Step 2-5: 다음 사용자 턴 시뮬레이션 ---
def simulate_next_user_turn(history: List[Dict], agent_final_text: str, next_utterance_type: str, max_retries: int = 1) -> Optional[str]:
    """LLM을 사용하여 다음 사용자 턴 발화 시뮬레이션."""
    print("\n--- 다음 턴: 시뮬레이션된 인간 ---")
    # **** 시스템 프롬프트 수정 ****
    system_prompt_human_next = f"""당신은 AI 비서와 대화 중인 사용자입니다. 이전 대화 내용과 방금 AI 비서가 한 말을 고려하여, 현재 대화 상황('{next_utterance_type}')에 가장 적절하고 **자연스러운 다음 사용자 응답**을 생성해주세요.
    - 비서의 답변에 대해 **적절히 반응**하세요 (예: 확인, 감사, 추가 질문).
    - 현재 상황 설명을 참고하여 대화를 **진행시키거나 마무리**하세요.
    - 너무 짧거나 단답형 응답보다는 자연스러운 문장으로 작성하세요.
    - 필요하다면 당신의 생각 과정을 <think>...</think> 태그 안에 먼저 작성할 수 있습니다."""
    user_prompt_human_next = f"대화 기록:\n{json.dumps(history, ensure_ascii=False)}\n\nAI 비서의 마지막 응답: '{agent_final_text}'\n\n'{next_user_utterance_type}' 상황에 맞는 당신의 다음 발화를 생성해주세요."

    api_response = call_friendli_api(messages=[{"role": "system", "content": system_prompt_human_next}, {"role": "user", "content": user_prompt_human_next}], role_description=f"시뮬레이션된 인간 ({next_user_utterance_type})", max_retries=max_retries)
    raw_utterance = api_response.get("content") if api_response else None
    # **** Think 태그 제거 로직 추가 ****
    user_utterance = remove_think_tags(raw_utterance)

    if user_utterance: print(f"다음 사용자 발화: {user_utterance}"); return user_utterance
    else: print("다음 사용자 발화 생성/파싱 실패."); return None # Think 태그만 있었던 경우 포함

# ---------------

# --- Step 2-6: 궤적 검증 ---
def validate_trajectory(executed_actions: List[ToolCallInternal], ground_truth_actions: List[ToolCallInternal]) -> bool:
    """실행된 도구 호출과 Ground Truth 비교하여 궤적 유효성 검증."""
    print("\n--- 2-6: 궤적 검증 ---")
    if len(executed_actions) == len(ground_truth_actions) and executed_actions == ground_truth_actions:
        print("궤적 검증 결과: 통과"); return True
    else:
        print(f"오류: 실행/GT 액션 불일치"); print(f"  GT: {[a.model_dump() for a in ground_truth_actions]}"); print(f"  Executed: {[e.model_dump() for e in executed_actions]}"); print("궤적 검증 결과: 실패"); return False

# ---------------

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    final_dataset = []
    num_data_to_generate = 1
    max_attempts_per_data = 3
    max_total_attempts = num_data_to_generate * max_attempts_per_data * 3

    seen_blueprint_ids = set()

    print("="*44); print(" APIGen-MT 데이터 생성 파이프라인 시작 (Refactored v6 - Prompt Update) "); print(f" 일반 모델: {MODEL_ID} / FC 모델: {FC_MODEL_ID}"); print(f" 생성 목표 데이터 수: {num_data_to_generate}"); print("="*44)

    generated_count = 0
    total_attempts_count = 0

    while generated_count < num_data_to_generate and total_attempts_count < max_total_attempts:
        total_attempts_count += 1
        print(f"\n\n===== 데이터 생성 시도 (목표: {generated_count + 1}/{num_data_to_generate}, 전체 시도: {total_attempts_count}) =====")
        current_data_generated_successfully = False

        for attempt in range(max_attempts_per_data):
            print(f"--- 내부 시도 {attempt + 1} / {max_attempts_per_data} ---")
            pipeline_failed_at_step = None

            # --- 1단계 ---
            blueprint_str = generate_blueprint_string(tools_definition_for_api)
            if not blueprint_str: pipeline_failed_at_step = "1-1"; continue
            validated_blueprint = parse_and_validate_blueprint(blueprint_str)
            if not validated_blueprint: pipeline_failed_at_step = "1-1b"; continue
            if not check_blueprint_execution(validated_blueprint, available_tools_mapping): pipeline_failed_at_step = "1-2"; continue
            passed_review, _ = review_blueprint(validated_blueprint)
            if not passed_review: pipeline_failed_at_step = "1-3"; continue

            # --- 청사진 중복 검사 ---
            blueprint_id = create_blueprint_id(validated_blueprint)
            if blueprint_id in seen_blueprint_ids:
                print(f"중복된 청사진 발견 (ID: {blueprint_id[:8]}...). 이번 시도 건너뜁니다.")
                pipeline_failed_at_step = "dedupe"
                continue

            # --- 2단계 ---
            print("=== 1단계 통과, 2단계 시작 ===")
            trajectory: List[Dict[str, Any]] = []
            history: List[Dict[str, Any]] = []
            executed_actions: List[ToolCallInternal] = []
            turn_count = 0; max_turns_traj = 8

            # **** 시스템 프롬프트 수정 ****
            system_msg_content = f"""당신은 사용자의 복잡한 요청을 처리하는 유능한 AI 에이전트입니다. 사용자와의 **다중 턴 대화**를 통해 사용자의 **숨겨진 의도나 필요한 세부 정보**를 파악하고, 목표를 달성하기 위해 적절한 시점에 다음 도구들을 사용할 수 있습니다. 사용자의 말을 주의 깊게 듣고, 필요한 정보가 부족하면 명확히 질문하세요. 도구 사용이 필요 없거나 모든 작업이 완료되면 사용자에게 **친절하고 자연스러운 언어**로 최종 답변을 제공하세요.
사용 가능한 도구: {', '.join(available_tools_mapping.keys())}. (세부 설명은 Function Calling 요청 시 제공됨)
오늘 날짜는 {datetime.now().strftime('%Y-%m-%d')}입니다. 대화 흐름을 관리하며 사용자의 문제를 해결하세요."""
            system_msg = {"role": "system", "content": system_msg_content}
            history.append(system_msg)

            # 2-1: 첫 사용자 턴
            raw_first_utterance = simulate_first_user_turn(validated_blueprint.q)
            # **** Think 태그 제거 로직 추가 **** (현재는 q를 바로 사용하므로 불필요)
            user_utterance = raw_first_utterance # remove_think_tags(raw_first_utterance) if using LLM
            if not user_utterance: pipeline_failed_at_step = "2-1"; break
            user_msg = {"role": "user", "content": user_utterance}
            history.append(user_msg); trajectory.append(user_msg)

            # 2-2 ~ 2-5: 대화 턴 루프
            while turn_count < max_turns_traj:
                turn_count += 1; print(f"\n--- Trajectory Turn {turn_count} ---")
                agent_response_message = simulate_agent_turn(history, tools_definition_for_api)
                if not agent_response_message: pipeline_failed_at_step = "2-2"; break

                trajectory.append(agent_response_message)
                sanitized_assistant_msg = {"role": agent_response_message.get("role", "assistant"), "content": agent_response_message.get("content")}
                if agent_response_message.get("tool_calls"): sanitized_assistant_msg["tool_calls"] = agent_response_message["tool_calls"]
                sanitized_assistant_msg = {k:v for k,v in sanitized_assistant_msg.items() if v is not None}
                history.append(sanitized_assistant_msg)

                agent_final_text = ""
                if agent_response_message.get("tool_calls"):
                    tool_calls = agent_response_message["tool_calls"]
                    tool_response_messages, successful_calls = execute_tool_calls(tool_calls, available_tools_mapping)
                    executed_actions.extend(successful_calls)
                    history.extend(tool_response_messages); trajectory.extend(tool_response_messages)
                    final_agent_response_message = get_final_agent_response(history, tools_definition_for_api)
                    if final_agent_response_message:
                         trajectory.append(final_agent_response_message)
                         sanitized_final_msg = {"role": final_agent_response_message.get("role", "assistant"),"content": final_agent_response_message.get("content")}
                         sanitized_final_msg = {k:v for k,v in sanitized_final_msg.items() if v is not None}
                         history.append(sanitized_final_msg)
                         agent_final_text = sanitized_final_msg.get("content", "")
                    else: pipeline_failed_at_step = "2-4"; break
                elif agent_response_message.get("content"): agent_final_text = agent_response_message.get("content", "")
                else: pipeline_failed_at_step = "agent_resp_error"; break

                if pipeline_failed_at_step: break

                if len(executed_actions) == len(validated_blueprint.a_gt) and not agent_response_message.get("tool_calls"): should_end_next = True; next_user_utterance_type = "감사 및 종료"
                elif len(executed_actions) < len(validated_blueprint.a_gt): should_end_next = False; next_user_utterance_type = "다음 질문 또는 확인"
                else: should_end_next = False; next_user_utterance_type = "대화 마무리"

                # **** Think 태그 제거 로직 추가 ****
                raw_next_utterance = simulate_next_user_turn(history, agent_final_text, next_user_utterance_type)
                next_user_utterance = remove_think_tags(raw_next_utterance)

                if not next_user_utterance: # None 또는 빈 문자열(Think만 있었던 경우)
                    print("2-5 실패 또는 빈 응답"); pipeline_failed_at_step = "2-5"; break

                is_ending = any(end in next_user_utterance for end in ["감사", "없습", "됐어", "그만", "수고", "알겠습니다", "해결됐", "괜찮"])
                user_msg = {"role": "user", "content": next_user_utterance}
                history.append(user_msg); trajectory.append(user_msg)
                if is_ending or should_end_next: print("대화 종료 조건 충족."); break
                if turn_count >= max_turns_traj - 1: print("최대 턴 도달."); break
            # --- 턴 루프 종료 ---

            if pipeline_failed_at_step: print(f"2단계 실패 (Step: {pipeline_failed_at_step}). 다음 시도 진행..."); continue

            # 2-6: 궤적 검증
            if validate_trajectory(executed_actions, validated_blueprint.a_gt):
                final_dataset.append({"blueprint": validated_blueprint.model_dump(), "trajectory": trajectory})
                seen_blueprint_ids.add(blueprint_id)
                generated_count += 1
                current_data_generated_successfully = True
                print(f"\n데이터 생성 성공! (현재 {generated_count}/{num_data_to_generate}개)")
                break # 현재 데이터 성공, 다음 데이터 생성으로
            else: print("궤적 검증 실패.") # 다음 내부 시도로

        # --- 내부 시도 루프 종료 ---
        if current_data_generated_successfully: continue
        else: print(f"내부 시도 {max_attempts_per_data}회 모두 실패 또는 중복.")

    # --- 최종 결과 처리 ---
    print(f"\n\n{'='*44}"); print(f" 총 {generated_count}개의 고유 데이터 생성 완료 (전체 시도 {total_attempts_count}회)."); print("="*44)
    if final_dataset:
        filename = f"apigen_mt_dataset_{MODEL_ID}_fc_{FC_MODEL_ID}_final_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"
        print(f"\n생성된 데이터를 '{filename}' 파일로 저장합니다.")
        try:
            with open(filename, "w", encoding="utf-8") as f:
                for data_item in final_dataset:
                    cleaned_trajectory = []
                    for turn in data_item['trajectory']:
                        cleaned_turn = {k: v for k, v in turn.items() if v is not None}
                        if 'tool_calls' in cleaned_turn and cleaned_turn['tool_calls']:
                            cleaned_tool_calls = []
                            for tc in cleaned_turn['tool_calls']:
                                cleaned_tc = {k: v for k, v in tc.items() if v is not None}
                                if 'function' in cleaned_tc and cleaned_tc['function']:
                                    cleaned_tc['function'] = {k: v for k, v in cleaned_tc['function'].items() if v is not None}
                                cleaned_tool_calls.append(cleaned_tc)
                            cleaned_turn['tool_calls'] = cleaned_tool_calls
                        cleaned_trajectory.append(cleaned_turn)
                    cleaned_data = {"blueprint": data_item['blueprint'], "trajectory": cleaned_trajectory}
                    f.write(json.dumps(cleaned_data, ensure_ascii=False) + "\n")
            print("파일 저장 완료.")
        except IOError as e: print(f"파일 저장 중 오류 발생: {e}")

 APIGen-MT 데이터 생성 파이프라인 시작 (Refactored v6 - Prompt Update) 
 일반 모델: deepseek-r1 / FC 모델: deepseek-r1
 생성 목표 데이터 수: 1


===== 데이터 생성 시도 (목표: 1/1, 전체 시도: 1) =====
--- 내부 시도 1 / 3 ---

--- 1-1: 작업 청사진 생성 (LLM 호출) ---

--- Friendli API 호출 (작업 생성 LLM) ---
사용 모델: deepseek-r1
성공: 응답 Role=assistant Content='<think>
먼저, 사용자의 요청과 도구들을 어떻게 결합할지 생각해야 해요. 두 가지 도...' 
성공: 청사진 JSON 문자열 추출 완료

--- 1-1b: 청사진 파싱 및 검증 (Pydantic) ---
성공: Blueprint 파싱 및 검증 완료

--- 1-2: 형식/실행 검사 ---
도구 'get_weather' (인자: {'location': 'Los Angeles', 'date': 'tomorrow'}) 실행 가능 확인.
오류: 'get_news' 인자 검증 실패: 1 validation error for GetNewsArgs
topic
  Field required [type=missing, input_value={'query': 'Los Angeles events'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
--- 내부 시도 2 / 3 ---

--- 1-1: 작업 청사진 생성 (LLM 호출) ---

--- Friendli API 호출 (작업 생성 LLM) ---
사용 모델: deepseek-r1
성공: 응답 Role=assistant Content='<think>
사용자가 날씨와 뉴스 정보를 결합하여 특정 날의 활동을 계획하는 시나리오를 ...' 
성공: 청사진 JSON 문자열 추출 